# Transport Efficiency

In [1]:
import pybamm
import numpy as np

In [2]:
sols = []
te_opts = pybamm.BatteryModelOptions({}).possible_options["transport efficiency"]
parameter_values = pybamm.ParameterValues("Marquis2019")
print(te_opts)

['Bruggeman', 'ordered packing', 'hyperbola of revolution', 'overlapping spheres', 'tortuosity factor', 'random overlapping cylinders', 'heterogeneous catalyst', 'cation-exchange membrane']


In [3]:
parameter_values.search("porosity")

Negative electrode porosity	0.3
Positive electrode porosity	0.3
Separator porosity	1.0


In [4]:
parameter_values.search("Bruggeman")

Negative electrode Bruggeman coefficient (electrode)	1.5
Negative electrode Bruggeman coefficient (electrolyte)	1.5
Positive electrode Bruggeman coefficient (electrode)	1.5
Positive electrode Bruggeman coefficient (electrolyte)	1.5
Separator Bruggeman coefficient (electrolyte)	1.5


Add tortuosity factors that replicate the Bruggeman values

In [5]:
parameter_values.update({'Negative electrode tortuosity factor (electrolyte)': 0.3**(-0.5),
                         'Positive electrode tortuosity factor (electrolyte)': 0.3**(-0.5),
                         'Negative electrode tortuosity factor (electrode)': 0.7**(-0.5),
                         'Positive electrode tortuosity factor (electrode)': 0.7**(-0.5),
                         'Separator tortuosity factor (electrolyte)': 1.0}, check_already_exists=False)

In [6]:
for t_label in te_opts:
    model = pybamm.lithium_ion.DFN(options={'transport efficiency': t_label})  # Doyle-Fuller-Newman model
    sim = pybamm.Simulation(model, parameter_values=parameter_values)
    sols.append(sim.solve([0, 3600]))  # solve for 1 hour

In [7]:
pybamm.dynamic_plot(sols,labels=te_opts)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [8]:
np.allclose(sols[0]["Terminal voltage [V]"].data, sols[4]["Terminal voltage [V]"].data)

True

In [9]:
parameter_values.update({'Negative electrode tortuosity factor (electrolyte)': 4.0,
                         'Positive electrode tortuosity factor (electrolyte)': 4.0,
                         'Negative electrode tortuosity factor (electrode)': 3.0,
                         'Positive electrode tortuosity factor (electrode)': 3.0,
                         'Separator tortuosity factor (electrolyte)': 1.5}, check_already_exists=False)

In [10]:
model = pybamm.lithium_ion.DFN(options={'transport efficiency': "tortuosity factor"})  # Doyle-Fuller-Newman model
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sols.append(sim.solve([0, 3600]))

In [11]:
pybamm.dynamic_plot(sols,labels=te_opts+["higher tortuosity factor"])

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [12]:
pybamm.print_citations()

[1] KA Akanni, JW Evans, and IS Abramson. Effective transport coefficients in heterogeneous media. Chemical Engineering Science, 42(8):1945–1954, 1987.
[2] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[3] JW Beeckman. Mathematical description of heterogeneous materials. Chemical engineering science, 45(8):2603–2610, 1990.
[4] Von DAG Bruggeman. Berechnung verschiedener physikalischer konstanten von heterogenen substanzen. i. dielektrizitätskonstanten und leitfähigkeiten der mischkörper aus isotropen substanzen. Annalen der physik, 416(7):636–664, 1935.
[5] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[6] Charles R